# Computing Very Large Correlation Matrices in Parallel

In [1]:
# data i/o
import os

# compute in parallel
from multiprocessing import Pool

# the usual
import numpy as np
import pandas as pd

import deepgraph as dg


Let's create a set variables and store it as a 2d-matrix ``X`` (``shape=(n_samples, n_features)``) on disc. To speed up the computation of the Pearson correlation coefficients later on, we whiten each variable.

In [42]:
# create observations
from numpy.random import RandomState
prng = RandomState(0)
n_samples = int(1e2)
n_features = int(1e1)
X = prng.randint(100, size=(n_samples, n_features)).astype(np.float64)

Xvar = X.var(axis=1, keepdims=True, ddof=1)
# whiten variables for fast parallel computation later on
X = X - X.mean(axis=1, keepdims=True)
#X = (X - X.mean(axis=1, keepdims=True)) / X.std(axis=1, keepdims=True)

# save in binary format
np.save('samples', X)


In [ ]:
# parameters (change these to control RAM usage)
step_size = 1e5
n_processes = 100

# load samples as memory-map
X = np.load('samples.npy', mmap_mode='r')

# create node table that stores references to the mem-mapped samples
v = pd.DataFrame({'index': range(X.shape[0])})

# connector function to compute pairwise pearson correlations
def corr(index_s, index_t):
    samples_s = X[index_s]
    samples_t = X[index_t]
    corr = np.einsum('ij,ij->i', samples_s, samples_t) / (n_features-1)
    return corr

# index array for parallelization
pos_array = np.array(np.linspace(0, n_samples*(n_samples-1)//2, n_processes), dtype=int)

# parallel computation
def create_ei(i):

    from_pos = pos_array[i]
    to_pos = pos_array[i+1]

    # initiate DeepGraph
    g = dg.DeepGraph(v)

    # create edges
    g.create_edges(connectors=corr, step_size=step_size, 
                   from_pos=from_pos, to_pos=to_pos)

    # store edge table
    g.e.to_pickle('/tmp/corr/{}.pickle'.format(str(i).zfill(3)))

# computation
if __name__ == '__main__':
    indices = np.arange(0, n_processes - 1)
    p = Pool()
    for _ in p.imap_unordered(create_ei, indices):
        pass
    

Let's collect the computed correlation values and store them in an hdf file.

In [32]:
# store correlation values
files = os.listdir('/tmp/corr/')
files.sort()
store = pd.HDFStore('e.h5', mode='w')
for f in files:
    et = pd.read_pickle('/tmp/corr/{}'.format(f))
    store.append('e', et, format='t', data_columns=True, index=False)
store.close()


In [23]:
test = np.cov(X)

In [44]:
test[1]

array([-128.16666667,  664.76666667,  -57.1       , -242.88888889,
        344.14444444,  198.06666667,  183.72222222, -223.92222222,
        -79.34444444, -696.75555556,  267.26666667,  -58.83333333,
          7.33333333,   10.11111111,  186.41111111,  282.25555556,
         60.18888889,  461.02222222, -419.5       ,  -13.44444444,
        -94.74444444, -116.77777778,   36.17777778,   54.5       ,
        139.48888889,  130.56666667, -232.64444444,  461.13333333,
        110.64444444, -179.08888889, -133.72222222, -118.07777778,
       -202.57777778,   52.38888889,  -41.24444444, -129.94444444,
         18.92222222, -259.22222222,  214.82222222, -221.32222222,
       -100.41111111, -106.51111111,  415.01111111,  278.74444444,
       -124.97777778,  442.15555556,   49.72222222, -109.37777778,
       -358.42222222,   -9.71111111, -125.24444444,  -80.18888889,
       -158.85555556, -189.27777778,  523.5       ,  -88.9       ,
        -42.85555556,  213.87777778,  338.2       ,  120.64444

Let's have a quick look at the correlations.

In [33]:
# load correlation table
e = pd.read_hdf('e.h5')
print(e)

             corr
s  t             
0  1  -128.166667
   2    93.500000
   3   136.333333
   4   -83.611111
   5    59.111111
   6  -128.722222
   7   -52.500000
   8     4.166667
   9   334.888889
   10 -239.222222
   11 -198.277778
   12 -421.555556
   13  215.666667
   14  -33.611111
   15   79.500000
   16  -17.388889
   17  268.777778
   18   70.388889
   19   58.222222
   20   10.166667
   21 -286.555556
   22 -108.888889
   23   63.611111
   24 -189.666667
   25   62.166667
   26  -81.666667
   27 -113.555556
   28  225.111111
   29    2.888889
   30 -319.944444
...           ...
91 98 -295.955556
   99 -377.488889
92 93   42.722222
   94 -138.633333
   95  180.100000
   96  435.355556
   97  151.900000
   98  156.566667
   99   13.877778
93 94 -437.833333
   95  238.833333
   96  117.222222
   97 -329.722222
   98  291.166667
   99   25.611111
94 95 -172.744444
   96 -322.066667
   97   77.522222
   98  172.188889
   99  107.255556
95 96  106.466667
   97  100.566667
   98  128

And finally, let's see where most of the computation time is spent.

In [11]:
g = dg.DeepGraph(v)
p = %prun -r g.create_edges(connectors=corr, step_size=step_size)

In [12]:
p.print_stats(20)

         7850 function calls (6521 primitive calls) in 0.013 seconds

   Ordered by: internal time
   List reduced from 506 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    202/4    0.001    0.000    0.003    0.001 abc.py:194(__subclasscheck__)
        2    0.001    0.000    0.001    0.000 {built-in method posix.stat}
        2    0.000    0.000    0.001    0.000 memmap.py:334(__getitem__)
      438    0.000    0.000    0.000    0.000 _weakrefset.py:70(__contains__)
 1221/728    0.000    0.000    0.004    0.000 {built-in method builtins.isinstance}
      186    0.000    0.000    0.001    0.000 _weakrefset.py:58(__iter__)
       47    0.000    0.000    0.000    0.000 tokenize.py:494(_tokenize)
   196/22    0.000    0.000    0.003    0.000 typing.py:1166(__subclasscheck__)
        2    0.000    0.000    0.000    0.000 internals.py:4801(_stack_arrays)
  508/118    0.000    0.000    0.003    0.000 {built-in method builtins.issubclas

As you can see, most of the time is spent by getting the requested samples in the corr-function, followed by computing the correlation values themselves. 